In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os

In [ ]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'
aztec_TEM_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec quant as EDS_TEM.xlsx'
df = pd.read_excel(aztec_TEM_file, sheet_name=1)
df.head(2)

In [ ]:
def quant_as_TEM(filename, a_corr=True, noise_slice=0.65, print_results=False):

    # thickness_list = [1e2, 1e3,1e4,1e5] # in nm
    thickness_list = [50, 100, 200, 400, 1e3, 1e4, 1e5] # in nm

    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    # if Vacc is lower than 20, slice at Vacc
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]

    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values
    kfactors = df_s['k-factor'].values
    intensities = s.get_lines_intensity()
    m_intensities = m.get_lines_intensity()

    # select only the relevant lines in m (relevant = we have a k-factor for it from AZtec)    
    lines_in_m_to_use = []
    for m_line in m_intensities:
        if m_line.metadata.Sample.xray_lines[0] in lines:
            lines_in_m_to_use.append(m_line)
    if a_corr:
    # calculate %at for each line in m, if possible
        for l in lines:
            try:
                m[l]
                for thickness in thickness_list:
                    at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
                    df_s[f'HS %at m A t={thickness:.0e}'] = [at_m[0][0].data.round(2)[0], at_m[0][1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s[f'HS %at m A t={thickness:.0e}'] = [np.nan, np.nan]

        # do the CL quantification with the intensities from the original spectrum
        at = s.quantification(intensities, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
        df_s['HS %at s A'] = [at[0][0].data.round(2)[0], at[0][1].data.round(2)[0]]
    
    else: # no absorption correction
        # almost the same code, but slicing at and at_m with one less level
        for l in lines:
            try:
                m[l]
                at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL')
                df_s['HS %at m'] = [at_m[0].data.round(2)[0], at_m[1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s['HS %at m'] = [np.nan, np.nan]
        at = s.quantification(intensities, factors=kfactors, method='CL')
        df_s['HS %at s'] = [at[0].data.round(2)[0], at[1].data.round(2)[0]]
            
    if print_results:
        print(df_s)

    # put results back in df
    df[df['File'] == filename] = df_s

In [ ]:
# testing
quant_as_TEM('GaAs_30kV_25pA',print_results=True)
#
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)
#
quant_as_TEM('GaAs_30kV_25pA',print_results=True)
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)

In [ ]:
# running the routine for all files in df
# took 3-4 min
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f)

In [ ]:
# took 30 sec
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f, a_corr=False)

In [ ]:
# save the df to excel
df.to_excel('results/quant_as_TEM.xlsx')

In [ ]:
fig = go.Figure()
lines_list = ['Sb_La', 'As_Ka', 'Ga_Ka', 'Ga_La', 'As_La']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] # plotly colors
          
y_keys = ['AZ %at', 'HS %at s','HS %at m A t=5e+01', 'HS %at m A t=1e+02', 'HS %at m A t=2e+02', 'HS %at m A t=4e+02', 'HS %at m A t=1e+03', 'HS %at m A t=1e+04', 'HS %at m A t=1e+05']

for l in lines_list:
    for y in y_keys:
        i = y_keys.index(y)
        fig.add_scatter(x=df[df['Line'] == l]['File'], y = df[df['Line'] == l][y], mode='markers', name=y, marker=dict(size=8, color=colors[i]), legendgroup=y, showlegend=True if l == lines_list[0] else False)
fig.update_layout(title='Data treated as TEM, quantified with HS using different quantification parameters<br><sub>s=original spectrum, m=model, A=absorption correction, t=thickness')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.update_yaxes(range=[0, 100])
fig
# fig.write_html('results/quant_as_TEM.html')
